In [1]:
!source ~/.bashrc

Removing conda version miniforge3-24.7.1
Loading conda version miniforge3-24.7.1


In [2]:
import os
hf_token = os.getenv('HF_TOKEN')
# cache_dir =  "/scratch3/workspace/wenlongzhao_umass_edu-reason/dev_kedar/transformers_cache"
# os.environ['TRANSFORMERS_CACHE'] = cache_dir
# os.environ['HF_HOME']=cache_dir
# os.environ['HF_HUB_CACHE']=cache_dir+'/hub'

In [2]:
import torch
from transformers import pipeline, AutoTokenizer
# import multiprocessing as mp

# mp.set_start_method("spawn", force=True)

from datasets import load_from_disk
from vllm.inputs import TokensPrompt
from vllm import LLM, SamplingParams
from tqdm import tqdm



/work/pi_mccallum_umass_edu/kchimmad_umass_edu/conda_envs/reason/lib/python3.12/site-packages/transformers/utils/hub.py:105: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [3]:
# Loading data
data_path = '/scratch3/workspace/wenlongzhao_umass_edu-reason/dev_kedar/Small-LLM-Reasoning/datasets/gsm8k/tokenized/Llama-3.2-1B-Instruct/test/0-shot'
# data_path= "../datasets/gsm8k/tokenized/Llama-3.2-1B-Instruct/"
# split='test'
# data=load_from_disk(data_path+split+'/0-shot')
data=load_from_disk(data_path)
# data = load_from_disk(f"{data_path}/tokenized/LLaMA8B-Instruct/{split}/0-shot/")
data

Dataset({
    features: ['question', 'answer', 'input_ids'],
    num_rows: 1319
})

In [2]:
# Loading model
hf_token = os.getenv("HF_TOKEN")

# # model_name= "meta-llama/Llama-3.2-1B-Instruct"
# # model_name='/datasets/ai/llama3/hub/models--meta-llama--Llama-3.2-1B-Instruct/snapshots/9213176726f574b556790deb65791e0c5aa438b6/'
# model_name='meta-llama/Llama-3.2-3B-Instruct'
# tokenizer = AutoTokenizer.from_pretrained(model_name, token=hf_token)
# tokenizer.pad_token_id = tokenizer.eos_token_id



In [4]:
print(os.getenv('HF_TOKEN'))

None


In [5]:
kwargs={
    'download_dir':cache_dir,
    'max_model_len': 2000
}

model = LLM(
        model=model_name,  
        tensor_parallel_size=1,
        trust_remote_code=True,
        # gpu_memory_utilization=0.97,  # Increase this value
    **kwargs
) 



NameError: name 'cache_dir' is not defined

In [6]:
stop_strings =  ['<|eot_id|>', '<|start_header_id|>user<|end_header_id|>', 'Q:', '</s>', '<|im_end|>', 'the']


sampling_params = SamplingParams(n=1,
                                 temperature=0,
                                 max_tokens=500,
                                 stop=stop_strings,
                                 # logprobs=2,
                                 best_of=1,
                                 presence_penalty=0,
                                 top_p=1,
                                 top_k=1
                                )

In [7]:
# import json
# task='gec'
# task_prompt_path=f'../prompts/{task}.json'
# with open(task_prompt_path) as fp:
#     task_prompt = json.load(fp)
# task_prompt

In [8]:
# prompt=[]
# for i in range(len(data)):
# # for i in range(26,28):
#     chat=[
#         {
#             "role":"system",
#             "content":task_prompt['system_msg'],
#         },
#         {
#             "role":"user",
#             "content":task_prompt['user_msg'].format(instruction=task_prompt['task_prompt'], question=data[i]['input'])
#         }
#     ]
#     prompt.append(tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True))
# # print(prompt)

In [9]:
# print(prompt[332])

In [21]:
all_outputs = model.generate(data['input_ids'][0], sampling_params)

Processed prompts: 100%|██████████| 1/1 [00:00<00:00, 15.77it/s, est. speed input: 2195.41 toks/s, output: 142.11 toks/s]


In [23]:
all_outputs[0].outputs[0]

CompletionOutput(index=0, text='To find out how much Janet makes at ', token_ids=array('l', [1271, 1505, 704, 1268, 1790, 54765, 3727, 520, 279]), cumulative_logprob=None, logprobs=None, finish_reason=stop, stop_reason=the)

In [24]:
all_outputs[0].outputs[0].text

'To find out how much Janet makes at '

In [26]:
tokenizer.decode(all_outputs[0].outputs[0].token_ids, skip_special_tokens=False)

'To find out how much Janet makes at the'

In [25]:
tokenizer.decode(all_outputs[0].outputs[0].token_ids, skip_special_tokens=True)

'To find out how much Janet makes at the'

In [27]:
all_outputs[0]

RequestOutput(request_id=1, prompt=None, prompt_token_ids=[128000, 128006, 9125, 128007, 271, 38766, 1303, 33025, 2696, 25, 6790, 220, 2366, 18, 198, 15724, 2696, 25, 220, 1187, 13806, 220, 2366, 20, 271, 128009, 128006, 882, 128007, 271, 22818, 279, 2768, 3575, 11, 2944, 323, 3041, 264, 1620, 4320, 311, 279, 3575, 627, 32298, 25, 54765, 753, 78878, 11203, 220, 845, 19335, 824, 1938, 13, 3005, 50777, 2380, 369, 17954, 1475, 6693, 323, 293, 2094, 55404, 1354, 369, 1077, 4885, 1475, 1938, 449, 3116, 13, 3005, 31878, 279, 27410, 520, 279, 20957, 6, 3157, 7446, 369, 400, 17, 824, 7878, 37085, 19151, 13, 2650, 1790, 304, 11441, 1587, 1364, 1304, 1475, 1938, 520, 279, 20957, 6, 3157, 5380, 7927, 2077, 1288, 842, 449, 330, 791, 1620, 4320, 374, 510, 9399, 19727, 1405, 510, 9399, 60, 374, 279, 2077, 311, 279, 3575, 13, 128009, 128006, 78191, 128007, 271], encoder_prompt=None, encoder_prompt_token_ids=None, prompt_logprobs=None, outputs=[CompletionOutput(index=0, text='To find out how much Jane

In [1]:
print('<|start_header_id|>user<|end_header_id|>\n\nWrite a solution to the following problem and make sure that it passes the tests:\n```python\nfrom typing import List\n\n\ndef has_close_elements(numbers: List[float], threshold: float) -> bool:\n \"\"\" Check if in given list of numbers, are any two numbers closer to each other than\n given threshold.\n >>> has_close_elements([1.0, 2.0, 3.0], 0.5)\n False\n >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)\n True\n \"\"\"\n\n```<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nHere is the completed function:\n```python\nfrom typing import List\n\n\ndef has_close_elements(numbers: List[float], threshold: float) -> bool:\n \"\"\" Check if in given list of numbers, are any two numbers closer to each other than\n given threshold.\n >>> has_close_elements([1.0, 2.0, 3.0], 0.5)\n False\n >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)\n True\n \"\"\"')


<|start_header_id|>user<|end_header_id|>

Write a solution to the following problem and make sure that it passes the tests:
```python
from typing import List


def has_close_elements(numbers: List[float], threshold: float) -> bool:
 """ Check if in given list of numbers, are any two numbers closer to each other than
 given threshold.
 >>> has_close_elements([1.0, 2.0, 3.0], 0.5)
 False
 >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)
 True
 """

```<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Here is the completed function:
```python
from typing import List


def has_close_elements(numbers: List[float], threshold: float) -> bool:
 """ Check if in given list of numbers, are any two numbers closer to each other than
 given threshold.
 >>> has_close_elements([1.0, 2.0, 3.0], 0.5)
 False
 >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)
 True
 """
